In [2]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
from typing import Literal, Optional

import pandas as pd
from entities import EM_CDS_TRACKER_DICT, FX_TRACKER_DICT
from utils import Backtest, load_trackers, clean_stale_prices
from pathlib import Path
from aamalha.utils.performance import Performance

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


# Trackers Statistics

In [ ]:
fx = load_trackers(FX_TRACKER_DICT).rename(columns=lambda col: col + "_fx")
cds = load_trackers(EM_CDS_TRACKER_DICT).rename(columns=lambda col: col + "_cds")

trackers = pd.concat(
    [fx, cds],
    axis=1,
).fillna(method="ffill")
trackers = pd.concat(
    [clean_stale_prices(trackers[col].copy()) for col in trackers], axis=1
)
trackers = trackers[:"2024-09-30"]
trackers

In [15]:
(trackers / trackers.apply(lambda col: col[col.first_valid_index()]) - 1).filter(
    like="_cds", axis=1
).sort_index(axis=1).rename(
    columns=lambda col: col[:3]
).to_clipboard(excel=True)

In [3]:
perf_trackers = Performance(trackers)

In [17]:
df_perf = perf_trackers.table.T.copy()
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
df_perf.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

In [ ]:
perf_trackers.rolling_sharpe.ARS_cds

In [5]:
DICT_COLS = {
    "count": "N obs",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}
trackers_rolling_sharpe = perf_trackers.rolling_sharpe.describe().T
trackers_rolling_sharpe["Início"] = perf_trackers.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
trackers_rolling_sharpe.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

# Valor

In [3]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []
for file_path in FOLDER.glob("*VALUE-*.xlsx"):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,VALUE-CDS-1,VALUE-CDS-12,VALUE-CDS-3,VALUE-CDS-6,VALUE-FX-1,VALUE-FX-12,VALUE-FX-3,VALUE-FX-6
date,,,,,,,,
2009-02-03,100.000000,NaN,100.000000,NaN,100.000000,NaN,100.000000,NaN
2009-02-04,99.932300,NaN,99.870016,NaN,101.409886,NaN,99.987692,NaN
2009-02-05,99.960182,NaN,99.979774,NaN,101.389313,NaN,99.018387,NaN
2009-02-06,100.066599,NaN,100.069889,NaN,101.734625,NaN,98.469117,NaN
2009-02-09,100.153646,NaN,100.006111,NaN,101.909729,NaN,98.939112,NaN
...,...,...,...,...,...,...,...,...
2024-09-24,40.101330,81.780428,60.591496,92.368321,106.006844,201.120002,103.845973,131.203978
2024-09-25,39.593416,81.020411,59.661797,91.540382,107.476198,203.157501,105.216536,132.695103
2024-09-26,39.879510,81.411450,60.102466,91.918861,107.358805,201.838780,104.992474,132.663733


In [4]:
from bwbbgdl import GoGet
import statsmodels.api as sm


tickers = {
    "SPXT Index": "S&P 500",
    "NDDUWI Index": "MSCI World",
    "FXCTEM8 Index": "FX Carry",
}
# tickers = ["SPXT Index", "FXCTEM8 Index", "MXWD Index", "HYG Equity", "EREM5LD5 Index"]
g = GoGet(enforce_strict_matching=True)
benchmarks: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
)
benchmarks = benchmarks.pivot_table(index="date", columns="id")
benchmarks.columns = benchmarks.columns.droplevel(0)
benchmarks = benchmarks.rename(columns=tickers)

N_DAYS = 1
base = pd.concat(
    [
        df_backtest["VALUE-FX-12"],
        benchmarks,
    ],
    axis=1,
    join="inner",
)
base = base.fillna(method="ffill")
base = base[~base["VALUE-FX-12"].isna()]
base = np.log(base).diff(N_DAYS).dropna()
endog = base["VALUE-FX-12"]
exog = base[base.columns.drop("VALUE-FX-12")]
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            VALUE-FX-12   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     61.76
Date:                Tue, 12 Nov 2024   Prob (F-statistic):           5.57e-39
Time:                        08:45:56   Log-Likelihood:                 13260.
No. Observations:                3788   AIC:                        -2.651e+04
Df Residuals:                    3784   BIC:                        -2.649e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.000      1.592      0.1

In [5]:
df_backtest

,VALUE-CDS-1,VALUE-CDS-12,VALUE-CDS-3,VALUE-CDS-6,VALUE-FX-1,VALUE-FX-12,VALUE-FX-3,VALUE-FX-6
date,,,,,,,,
2009-02-03,100.000000,NaN,100.000000,NaN,100.000000,NaN,100.000000,NaN
2009-02-04,99.932300,NaN,99.870016,NaN,101.409886,NaN,99.987692,NaN
2009-02-05,99.960182,NaN,99.979774,NaN,101.389313,NaN,99.018387,NaN
2009-02-06,100.066599,NaN,100.069889,NaN,101.734625,NaN,98.469117,NaN
2009-02-09,100.153646,NaN,100.006111,NaN,101.909729,NaN,98.939112,NaN
...,...,...,...,...,...,...,...,...
2024-09-24,40.101330,81.780428,60.591496,92.368321,106.006844,201.120002,103.845973,131.203978
2024-09-25,39.593416,81.020411,59.661797,91.540382,107.476198,203.157501,105.216536,132.695103
2024-09-26,39.879510,81.411450,60.102466,91.918861,107.358805,201.838780,104.992474,132.663733


## Backtests

In [22]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [23]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [24]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [26]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [25]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [27]:
# rolling_sharpe_quantiles

rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N +1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Only

In [29]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "L-CDS-EW",
    "L-CDS-IV",
    "L-FX-EW",
    "L-FX-IV",
    "L-EW",
    "L-IV",
]


In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "L-CDS-EW",
    "L-CDS-IV",
    "L-FX-EW",
    "L-FX-IV",
    "L-EW",
    "L-IV",
]
file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in file_pathS:
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [51]:
df_backtest.to_clipboard(excel=True)


In [ ]:
df_backtest

## Backtests

In [37]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [50]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [49]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [47]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [45]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [44]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N +1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Short- Classe

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)

file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in list(FOLDER.glob("L-*-S-*")):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [57]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [58]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [59]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [60]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [61]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [62]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [63]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Short - Ativo

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "LS-CDS-FX-BN-IV",
    "LS-FX-CDS-BN-IV",
    "LS-FX-CDS-IV",
    "LS-FX-CDS-IV",
]
file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in file_pathS:
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [65]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [ ]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [ ]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [ ]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

# TSMOM

In [3]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []
for file_path in FOLDER.glob("*TSMOM-*.xlsx"):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,TSMOM-1,TSMOM-12,TSMOM-3,TSMOM-6,TSMOM-CDS-1,TSMOM-CDS-12,TSMOM-CDS-3,TSMOM-CDS-6,TSMOM-FX-1,TSMOM-FX-12,TSMOM-FX-3,TSMOM-FX-6
date,,,,,,,,,,,,
2009-02-03,599.385387,405.953149,583.318560,437.893399,108.569026,134.613326,125.581494,101.583481,654.423961,363.245445,545.997702,456.702324
2009-02-04,601.189634,406.249693,581.866063,438.213276,109.089778,134.954374,125.243590,101.840846,655.005251,362.916313,546.314524,456.288511
2009-02-05,601.475296,406.647823,579.939095,438.642730,109.523981,135.437973,125.019813,102.205785,653.216988,362.414081,544.863482,455.657064
2009-02-06,598.960692,405.000589,580.881745,436.865892,108.948569,134.885730,125.535937,101.789045,651.603047,361.092999,542.183663,453.996090
2009-02-09,596.436886,403.192020,580.034364,434.915026,108.780375,134.571199,125.927030,101.551690,647.795160,358.957844,537.465387,451.311595
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24,814.705461,304.769044,594.304749,491.787374,129.659723,109.058280,169.147778,115.160281,665.543296,227.400326,440.312178,334.207107
2024-09-25,816.828903,305.321037,598.648534,494.224229,129.534642,108.953073,169.504970,115.258729,669.822990,229.696011,444.438578,337.246686
2024-09-26,821.954881,306.711307,600.783502,497.033337,130.400602,109.681442,170.384206,115.981474,672.047212,228.377767,444.989636,337.850960


In [4]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [5]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [6]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [9]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [7]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [8]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()


In [11]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# XSMOM

In [12]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []
for file_path in FOLDER.glob("*XSMOM-*.xlsx"):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,XSMOM-1,XSMOM-12,XSMOM-3,XSMOM-6,XSMOM-CDS-1,XSMOM-CDS-12,XSMOM-CDS-3,XSMOM-CDS-6,XSMOM-FX-1,XSMOM-FX-12,XSMOM-FX-3,XSMOM-FX-6
date,,,,,,,,,,,,
2009-02-03,282.402450,250.346763,270.757717,275.222275,141.649592,166.350335,140.005977,142.747182,299.225495,228.620483,255.448662,246.046801
2009-02-04,283.497366,249.358678,271.503474,274.559749,142.653727,165.712605,140.501392,142.461973,300.476350,228.636447,256.648434,246.662504
2009-02-05,282.000841,248.694883,270.380435,273.783401,142.311989,165.961493,140.266817,142.597027,299.499472,227.778693,255.906597,245.929622
2009-02-06,281.385949,248.206355,269.910005,272.955908,141.930179,166.290582,140.309475,142.666301,299.031579,226.385891,255.190169,244.417067
2009-02-09,279.357451,245.822921,268.057541,270.243398,142.225829,165.969603,140.657103,142.364383,295.587666,222.631113,251.753742,240.328451
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24,330.211571,159.372403,209.638028,431.244489,105.605349,153.913355,158.675570,350.636584,443.977576,117.993861,237.756743,310.412286
2024-09-25,333.512591,160.365914,213.015290,437.923036,105.645959,153.949540,159.655688,350.929146,448.628292,118.724311,241.401986,314.512885
2024-09-26,333.494003,159.541540,212.236958,438.490635,105.674203,153.956039,159.748693,351.009988,447.887851,118.047026,240.534652,315.059225


In [14]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [13]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [15]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [16]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [18]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [17]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()


In [19]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Performance Comparada

In [4]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "L-CDS-S-FX-EW",
    "XSMOM-CDS-6",
    "VALUE-FX-12",
    "XSMOM-FX-1",
    "L-CDS-EW",
    "TSMOM-CDS-3",
    "TSMOM-FX-1",
    "VALUE-CDS-6",
]
file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in file_pathS:
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,L-CDS-S-FX-EW,XSMOM-CDS-6,VALUE-FX-12,XSMOM-FX-1,L-CDS-EW,TSMOM-CDS-3,TSMOM-FX-1,VALUE-CDS-6
date,,,,,,,,
2009-02-03,93.972291,142.747182,NaN,299.225495,75.601524,125.581494,654.423961,NaN
2009-02-04,93.881292,142.461973,NaN,300.476350,75.519193,125.243590,655.005251,NaN
2009-02-05,93.707908,142.597027,NaN,299.499472,75.293170,125.019813,653.216988,NaN
2009-02-06,93.525392,142.666301,NaN,299.031579,75.533589,125.535937,651.603047,NaN
2009-02-09,93.149847,142.364383,NaN,295.587666,75.724180,125.927030,647.795160,NaN
...,...,...,...,...,...,...,...,...
2024-09-24,341.431973,350.636584,201.120002,443.977576,104.085365,169.147778,665.543296,92.368321
2024-09-25,340.683072,350.929146,203.157501,448.628292,104.092511,169.504970,669.822990,91.540382
2024-09-26,342.192446,351.009988,201.838780,447.887851,104.229231,170.384206,672.047212,91.918861


In [23]:
import scipy.cluster.hierarchy as sch

corr_matrix = np.log(df_backtest).diff(21).corr()
distance_matrix = 1 - corr_matrix
linkage_matrix = sch.linkage(distance_matrix, method="ward")
dendro_order = sch.leaves_list(linkage_matrix)
reordered_corr_matrix = corr_matrix.iloc[dendro_order, dendro_order]
reordered_corr_matrix.to_clipboard(excel=True)

C:\Users\pcampos\AppData\Local\Temp\ipykernel_2572\531783296.py:9: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = sch.linkage(distance_matrix, method="ward")


In [ ]:
df_backtest.to_clipboard(excel=True)


In [10]:
df_backtest.drop("VALUE-FX-12", axis=1)

,L-CDS-S-FX-EW,XSMOM-CDS-6,XSMOM-FX-1,L-CDS-EW,TSMOM-CDS-3,TSMOM-FX-1,VALUE-CDS-6
date,,,,,,,
2009-02-03,93.972291,142.747182,299.225495,75.601524,125.581494,654.423961,NaN
2009-02-04,93.881292,142.461973,300.476350,75.519193,125.243590,655.005251,NaN
2009-02-05,93.707908,142.597027,299.499472,75.293170,125.019813,653.216988,NaN
2009-02-06,93.525392,142.666301,299.031579,75.533589,125.535937,651.603047,NaN
2009-02-09,93.149847,142.364383,295.587666,75.724180,125.927030,647.795160,NaN
...,...,...,...,...,...,...,...
2024-09-24,341.431973,350.636584,443.977576,104.085365,169.147778,665.543296,92.368321
2024-09-25,340.683072,350.929146,448.628292,104.092511,169.504970,669.822990,91.540382
2024-09-26,342.192446,351.009988,447.887851,104.229231,170.384206,672.047212,91.918861


In [5]:
df_backtest["VALUE-FX-12"].ewm(halflife=252).corr(df_backtest.drop("VALUE-FX-12", axis=1)).dropna(how="all").to_clipboard()

## Backtests

In [24]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [25]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [ ]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()


In [ ]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()